# Introduction to plotly.py
A basic intro to interactive plotting with the plotly python package.  
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/Plotly_logo_for_digital_final_%286%29.png/800px-Plotly_logo_for_digital_final_%286%29.png?20210106093240" width=100 height=100>

## What is plotly?
* A python/R/MATLAB/Javascript API to the plotly.js library
* Easily create interactive plots
* Great for data exploration

### API levels in python
1) plotly figure factory
2) plotly express
3) plotly graph object
4) Dash

## Installing plotly
Easiest way - conda/mamba:  
`conda install -c conda-forge plotly`

## Example dataset - Titanic
<img src="https://media.istockphoto.com/vectors/the-big-passenger-ship-collided-with-an-iceberg-and-sank-into-the-sea-vector-id1337405629" width=400 height=400>

In [ ]:
import pandas as pd

In [ ]:
titanic_df = pd.read_csv('titanic.csv', index_col=0)
titanic_df

## Quick plotting with plotly express

In [ ]:
import plotly.express as px

### Simple plot

In [ ]:
fig = px.histogram(data_frame=titanic_df, x='Age')
fig.show()

### Useful options

In [ ]:
fig = px.histogram(data_frame=titanic_df, x='Age', title='Histogram of passenger age',
                   color_discrete_sequence=['black'], histnorm='percent')
fig.show()

### Changing the layout

In [ ]:
fig = px.histogram(data_frame=titanic_df, x='Age', title='Histogram of passenger age',
                   color_discrete_sequence=['black'], histnorm='percent')
fig.update_layout(xaxis_title_text='Age (years)', yaxis_title_text='%')
fig.update_yaxes(showgrid=False)
fig.show()

### Multiple histograms - same plot

In [ ]:
fig = px.histogram(data_frame=titanic_df, x='Age', color='Pclass',
                  barmode='overlay', opacity=0.5)
fig.show()

### Multiple histograms - facets (subplots)

In [ ]:
fig = px.histogram(data_frame=titanic_df, x='Age', facet_col='Pclass', histnorm='percent',
                  category_orders={'Pclass': [1,2,3]})
fig.show()

### Another example - box plots

In [ ]:
fig = px.box(titanic_df, x='Pclass', y="Age", color="Survived", facet_row='Sex')
fig.show()

## Customized plotting with plotly graph objects

In [ ]:
import plotly.graph_objects as go

### A simple histogram with go

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=titanic_df['Age']))
fig.show()

### Adding more traces

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=titanic_df['Age'], opacity=0.5, name='Age'))
fig.add_trace(go.Histogram(x=titanic_df['Fare'], opacity=0.5, name='Fare'))
fig.update_layout(barmode='overlay')
fig.show()

### Arbitrary subplots

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Histogram(x=titanic_df['Age']), row=1, col=1)
fig.add_trace(go.Box(x=titanic_df['Pclass'], y=titanic_df['Age']), row=1, col=2)
fig.update_layout(showlegend=False)
fig.show()

### MSA diversity

In [ ]:
from Bio import AlignIO

In [ ]:
def msa_to_df(msa):
    seq = []
    for rec in msa:
        seq_ser = pd.Series(list(rec.seq), name=rec.id)
        seq.append(seq_ser)
    return pd.concat(seq, axis=1)

In [ ]:
def msa_plot(msa_p, ignore_gaps=False):
    msa = AlignIO.read(msa_p, 'fasta')
    msa_df = msa_to_df(msa)
    freq_df = msa_df.apply(lambda row: row.value_counts(), axis=1).fillna(0)
    residues = ['A','T','G','C','-']
    if ignore_gaps:
        residues = ['A','T','G','C']
    freq_df = freq_df[residues]
    frac_df = freq_df.div(freq_df.sum(axis=1), axis=0)
    pos_diversity = 1 - frac_df.max(axis=1)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[50,100])
    fig.add_trace(go.Scatter(x=pos_diversity.index, y=pos_diversity, mode='lines', marker_color='black', name='Nucleotide diversity'),
              col=1, row=1)
    for res in residues:
        fig.add_trace(go.Bar(x=frac_df.index, y=frac_df[res], name=res), col=1, row=2)
    fig.update_layout(barmode='stack', bargap=0)
    fig['layout']['xaxis2']['title']='Position'
    return fig

In [ ]:
fig = msa_plot('16SRNA_Deino_87seq.aln')
fig.show()

## Themes
Plotly has a bunch of layout themes.

In [ ]:
import plotly.io as pio
pio.templates

In [ ]:
fig.update_layout(template='plotly_white')
fig.show()

In [ ]:
fig.update_layout(template='ggplot2')
fig.show()

## Exporting figures

### To static image

In [ ]:
# To png
fig.write_image('MSA.png')
# To pdf
fig.write_image('MSA.pdf')

### To HTML

In [ ]:
fig.write_html('MSA.html')

### The whole notebook to HTML

In [ ]:
!jupyter nbconvert intro_to_plotly.ipynb --to html --execute

## Widgets
Plotly has built-in options for buttons, sliders, checkboxes etc, providing high levels of control.  
But in many cases it is easier to use ipywidgets.

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact, fixed

In [ ]:
interact(msa_plot, msa_p=fixed('16SRNA_Deino_87seq.aln'), ignore_gaps=False)